# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 72067.08it/s]


In [6]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [7]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [8]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [9]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[ 4.5220e-01, -8.0138e-01, -2.6229e+00,  ..., -1.9492e+00,
          -9.5384e-01, -3.1468e+00],
         [-7.3974e-02, -2.4676e-01,  1.7123e+00,  ...,  3.1493e-01,
          -2.8493e+00,  6.1556e-01],
         [-9.8629e-01, -1.6576e+00, -9.9501e-02,  ...,  1.1769e+00,
           1.3054e+00,  1.6423e+00],
         ...,
         [ 7.2711e-01, -8.5266e-01, -2.5614e-01,  ...,  2.6862e-01,
          -1.4141e+00,  1.8307e+00],
         [ 7.2711e-01, -8.5266e-01, -2.5614e-01,  ...,  2.6862e-01,
          -1.4141e+00,  1.8307e+00],
         [ 7.2711e-01, -8.5266e-01, -2.5614e-01,  ...,  2.6862e-01,
          -1.4141e+00,  1.8307e+00]],

        [[ 1.9200e+00,  6.2438e-01,  2.7738e-01,  ..., -1.0504e+00,
           4.2086e-01,  8.0782e-01],
         [-6.7074e-01, -7.6778e-01,  1.3423e+00,  ...,  8.1818e-01,
           7.4122e-01,  1.0891e-01],
         [-8.1829e-01, -2.1970e-03, -5.6837e-02,  ...,  3.8242e-01,
           4.5298e-01,  2.9236e-02],
         ...,
         [ 7.2711e-01, -8

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [11]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [12]:
w_0 = nn.Linear(d_model, d_model)

In [13]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [14]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [15]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [16]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0586, 0.0460, 0.0470,  ..., 0.0554, 0.0554, 0.0554],
          [0.0505, 0.0570, 0.0441,  ..., 0.0416, 0.0416, 0.0416],
          [0.0572, 0.0612, 0.0539,  ..., 0.0529, 0.0529, 0.0529],
          ...,
          [0.0496, 0.0508, 0.0442,  ..., 0.0547, 0.0547, 0.0547],
          [0.0496, 0.0508, 0.0442,  ..., 0.0547, 0.0547, 0.0547],
          [0.0496, 0.0508, 0.0442,  ..., 0.0547, 0.0547, 0.0547]],

         [[0.0611, 0.0447, 0.0514,  ..., 0.0426, 0.0426, 0.0426],
          [0.0454, 0.0496, 0.0961,  ..., 0.0338, 0.0338, 0.0338],
          [0.0365, 0.0651, 0.0539,  ..., 0.0337, 0.0337, 0.0337],
          ...,
          [0.0921, 0.0486, 0.0752,  ..., 0.0461, 0.0461, 0.0461],
          [0.0921, 0.0486, 0.0752,  ..., 0.0461, 0.0461, 0.0461],
          [0.0921, 0.0486, 0.0752,  ..., 0.0461, 0.0461, 0.0461]],

         [[0.0557, 0.0457, 0.0426,  ..., 0.0321, 0.0321, 0.0321],
          [0.0260, 0.0547, 0.0507,  ..., 0.0363, 0.0363, 0.0363],
          [0.0490, 0.0620, 0.0292,  ..., 0

In [17]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [18]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [19]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0498, -0.1877,  0.0451,  ..., -0.1121,  0.0017,  0.0561],
         [ 0.1213, -0.1669, -0.0111,  ..., -0.1209, -0.0008,  0.0905],
         [ 0.0113, -0.1937,  0.0712,  ..., -0.1122, -0.0188,  0.0338],
         ...,
         [ 0.0945, -0.2164,  0.0144,  ..., -0.1014, -0.0286,  0.0754],
         [ 0.0945, -0.2164,  0.0144,  ..., -0.1014, -0.0286,  0.0754],
         [ 0.0945, -0.2164,  0.0144,  ..., -0.1014, -0.0286,  0.0754]],

        [[-0.0921, -0.0291,  0.0240,  ..., -0.1055,  0.0453,  0.1640],
         [-0.0740, -0.0350,  0.0380,  ..., -0.0870,  0.0439,  0.2407],
         [-0.0305, -0.0278,  0.0313,  ..., -0.1255, -0.0081,  0.1881],
         ...,
         [-0.0527, -0.0106,  0.0771,  ..., -0.0947,  0.0251,  0.1803],
         [-0.0527, -0.0106,  0.0771,  ..., -0.0947,  0.0251,  0.1803],
         [-0.0527, -0.0106,  0.0771,  ..., -0.0947,  0.0251,  0.1803]],

        [[ 0.0749, -0.0447, -0.1285,  ..., -0.0682,  0.1201,  0.1210],
         [ 0.0324,  0.0119, -0.0381,  ..., -0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [29]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)
    
    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = q.view(batch_size, -1, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)


    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [30]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [31]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[ 0.0419, -0.0112,  0.1490,  ..., -0.1647,  0.0510, -0.0326],
         [ 0.0678,  0.0566,  0.1247,  ..., -0.2711,  0.0454, -0.0792],
         [ 0.0473, -0.0124,  0.1608,  ..., -0.1406,  0.0541, -0.0815],
         ...,
         [ 0.0184, -0.0350,  0.1350,  ..., -0.1821,  0.1410, -0.1443],
         [ 0.0184, -0.0350,  0.1350,  ..., -0.1821,  0.1410, -0.1443],
         [ 0.0184, -0.0350,  0.1350,  ..., -0.1821,  0.1410, -0.1443]],

        [[ 0.3123,  0.0116,  0.2712,  ..., -0.2521,  0.3970, -0.0357],
         [ 0.2142, -0.0959,  0.2700,  ..., -0.2405,  0.3860, -0.0278],
         [ 0.2883, -0.0547,  0.2722,  ..., -0.2167,  0.3707,  0.0013],
         ...,
         [ 0.2162, -0.0409,  0.2799,  ..., -0.2539,  0.4262, -0.0722],
         [ 0.2162, -0.0409,  0.2799,  ..., -0.2539,  0.4262, -0.0722],
         [ 0.2162, -0.0409,  0.2799,  ..., -0.2539,  0.4262, -0.0722]],

        [[ 0.2389, -0.0675,  0.1496,  ..., -0.1380,  0.1486,  0.0079],
         [ 0.2673,  0.0324,  0.1851,  ..., -0